# Import Libraries

In [123]:
# Standard library imports
import os # allows access to OS-dependent functionalities
import sys # to manipulate different parts of the Python runtime environment
import math
import csv

# Libraries go get shop information
import requests  # for HTTP requests
from bs4 import BeautifulSoup # for HTML scrapping 

# Libraries to Generate random drivers information
import random
import string

import pandas as pd
import folium

# setting display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Get the current working directory
cwd = os.getcwd()

# Add the path of the utils directory to sys.path
utils_path = os.path.abspath(os.path.join(cwd, '..', 'utils'))
sys.path.append(utils_path)

#Preparing folder variables

main_folder = os.path.abspath(os.path.join(os.pardir))
data_folder = (main_folder + "/" +"data")
saved_models_folder = (data_folder + "/" + "saved_models")
raw_data = (data_folder + "/" + "_raw")
processed_data = (data_folder + "/" + "processed")
content_based_supervised_data = (main_folder + "/" + "processed" + "/" + "content_based_supervised")

# Generating random drivers information about location

In [156]:
import random
import string

madrid_lat_bounds = (40.280623, 40.494425) # latitude bounds of Madrid
madrid_lon_bounds = (-3.889459, -3.537076) # longitude bounds of Madrid

drivers_location = []

#  generate 50 records with driver_id, lat, and lon values
for i in range(1, 51):
    driver_id = ''.join(random.choice('0123456789ABCDEF') for j in range(4)) # generate 4 character random id
    lat = round(random.uniform(madrid_lat_bounds[0], madrid_lat_bounds[1]), 6)
    lon = round(random.uniform(madrid_lon_bounds[0], madrid_lon_bounds[1]), 6)
    disponibility = random.choice([True, False]) # randomly assign True or False to disponibility
    drivers_location.append({'driver_id': driver_id, 'lat': lat, 'lon': lon, 'disponibility': disponibility})


# open a CSV file in write mode
with open(raw_data + "/" + 'drivers_location.csv', mode='w', newline='') as file:

    # create a writer object
    writer = csv.writer(file)

    # write the header row
    writer.writerow(['driver_id', 'lat', 'lon','disponibility'])

    # write the data rows
    for address in drivers_location:
        writer.writerow([address['driver_id'], address['lat'], address['lon'], address['disponibility']])

In [157]:
# read the CSV file into a DataFrame
df_drivers = pd.read_csv(raw_data + "/" + "drivers_location.csv")

In [160]:
df_drivers.head(5)

,driver_id,lat,lon,disponibility
0,A8EB,40.291556,-3.629250,True
1,DFC5,40.339510,-3.741539,True
2,B13D,40.448745,-3.693041,False
3,ACA2,40.349852,-3.612646,True
4,32C6,40.300321,-3.716476,True


# Getting Shops information using Foursquare API

In [120]:
import requests
category_id = 13000
lat = 40.41517
lng = -3.71273
limit = 50

url = "https://api.foursquare.com/v3/places/search?ll={}%2C{}&radius={}&categories={}&limit={}".format(
            lat, 
            lng,
            radius,
            category_id,
            limit)

headers = {
    "accept": "application/json",
    "Authorization": "fsq3/yhR0cDdx5Y/JGfd12cmI90pn5vnosARY51e1sPopbU="
}

response = requests.get(url, headers=headers).json()

#Creating the necessary lists
places_list = []
# Extract the relevant information from the response
places = response['results']
for place in places:
    places_dict = {"id" : place["fsq_id"],
                   "name" : place["name"],
                   "category" : place["categories"][-1]["name"],
                   "formatted_address" : place["location"]["formatted_address"],
                   "latitude" : place["geocodes"]["main"]["latitude"],
                   "longitude" : place["geocodes"]["main"]["longitude"],
                   "admin_region" : place["location"]["admin_region"],
                   "locality" : place["location"]["locality"],
                   "region" : place["location"]["region"]

                    }
    places_list.append(places_dict) # Append the loop info to anime_list
places_list

[{'id': '4c321c9816adc9281693c19c',
  'name': 'La Mayor',
  'category': 'Tapas Restaurant',
  'formatted_address': 'Calle Mayor, 77 (C. Bailén), 28013 Madrid Madrid',
  'latitude': 40.415106,
  'longitude': -3.712051,
  'admin_region': 'Comunidad de Madrid',
  'locality': 'Madrid',
  'region': 'Madrid'},
 {'id': '593be9b516fa0435e49bbc2f',
  'name': 'Zúccaru',
  'category': 'Italian Restaurant',
  'formatted_address': 'Calle Vergara, 16, 28013 Madrid Madrid',
  'latitude': 40.417227,
  'longitude': -3.711617,
  'admin_region': 'Comunidad de Madrid',
  'locality': 'Madrid',
  'region': 'Madrid'},
 {'id': '4adcda36f964a5208b3b21e3',
  'name': 'Corral de la Morería',
  'category': 'Spanish Restaurant',
  'formatted_address': 'Calle Morería, 17, 28005 Madrid Madrid',
  'latitude': 40.412736,
  'longitude': -3.714198,
  'admin_region': 'Comunidad de Madrid',
  'locality': 'Madrid',
  'region': 'Madrid'},
 {'id': '4f6b0ec86b74877569ff1026',
  'name': 'La Rebelión de los Mandiles',
  'categor

In [127]:
# open a CSV file in write mode
with open(raw_data + "/" + 'shops_location.csv', mode='w', newline='') as file:

    # create a writer object
    writer = csv.writer(file)

    # write the header row
    writer.writerow(['id', 'name', 'category','formatted_address','latitude','longitude','admin_region','locality','region'])

    # write the data rows
    for place in places_list:
        writer.writerow([place['id'], place['name'], place['category'], place['formatted_address'], place['latitude'], place['longitude'], place['admin_region'], place['locality'], place['region']])

In [129]:
# read the CSV file into a DataFrame
df_shops = pd.read_csv(raw_data + "/" + "shops_location.csv")
df_shops.head(2)

,id,name,category,formatted_address,latitude,longitude,admin_region,locality,region
0,4c321c9816adc9281693c19c,La Mayor,Tapas Restaurant,"Calle Mayor, 77 (C. Bailén), 28013 Madrid Madrid",40.415106,-3.712051,Comunidad de Madrid,Madrid,Madrid
1,593be9b516fa0435e49bbc2f,Zúccaru,Italian Restaurant,"Calle Vergara, 16, 28013 Madrid Madrid",40.417227,-3.711617,Comunidad de Madrid,Madrid,Madrid


# Practizing location

In [161]:
# read the drivers_location CSV file into a DataFrame 
df_drivers = pd.read_csv(raw_data + "/" + "drivers_location.csv")
df_drivers.head(5)

,driver_id,lat,lon,disponibility
0,A8EB,40.291556,-3.629250,True
1,DFC5,40.339510,-3.741539,True
2,B13D,40.448745,-3.693041,False
3,ACA2,40.349852,-3.612646,True
4,32C6,40.300321,-3.716476,True


In [168]:
df_shops = pd.read_csv(raw_data + "/" + "shops_location.csv")
df_shops.head()

,id,name,category,formatted_address,latitude,longitude,admin_region,locality,region
0,4c321c9816adc9281693c19c,La Mayor,Tapas Restaurant,"Calle Mayor, 77 (C. Bailén), 28013 Madrid Madrid",40.415106,-3.712051,Comunidad de Madrid,Madrid,Madrid
1,593be9b516fa0435e49bbc2f,Zúccaru,Italian Restaurant,"Calle Vergara, 16, 28013 Madrid Madrid",40.417227,-3.711617,Comunidad de Madrid,Madrid,Madrid
2,4adcda36f964a5208b3b21e3,Corral de la Morería,Spanish Restaurant,"Calle Morería, 17, 28005 Madrid Madrid",40.412736,-3.714198,Comunidad de Madrid,Madrid,Madrid
3,4f6b0ec86b74877569ff1026,La Rebelión de los Mandiles,Deli,"Calle Mayor, 88, 28013 Madrid Madrid",40.415218,-3.713306,Comunidad de Madrid,Madrid,Madrid
4,4b953cc5f964a5200f9734e3,Restaurante Taberneros,Spanish Restaurant,"Santiago, 9, 28013 Madrid Madrid",40.416388,-3.710516,Comunidad de Madrid,Madrid,Madrid


In [172]:
df_drivers = pd.read_csv(raw_data + "/" + "drivers_location.csv")
df_drivers.columns

Index(['driver_id', 'lat', 'lon', 'disponibility'], dtype='object')

In [173]:
df_shops = pd.read_csv(raw_data + "/" + "shops_location.csv")
df_shops.columns

Index(['id', 'name', 'category', 'formatted_address', 'latitude', 'longitude',
       'admin_region', 'locality', 'region'],
      dtype='object')

In [174]:
import numpy as np
from math import radians, cos, sin, asin, sqrt

# read the drivers_location CSV file into a DataFrame 
df_drivers = pd.read_csv(raw_data + "/" + "drivers_location.csv")

# Convert a Pandas DataFrame df_drivers to a list of dictionaries
driver_dicts = df_drivers.to_dict(orient='records')

# read the shops_location CSV file into a DataFrame
df_shops = pd.read_csv(raw_data + "/" + "shops_location.csv")

# Convert a Pandas DataFrame df_shops to a list of dictionaries
shops_dicts = df_shops.to_dict(orient='records')

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def find_closest_driver(df_drivers, df_shops, shop_name):
    # Get the latitude and longitude of the specified shop
    shop_location = df_shops.loc[df_shops['name'] == shop_name, ['latitude', 'longitude']].values.flatten()

    # Calculate the distance between each driver's location and the shop location using the Haversine formula
    distances = np.array([haversine(row['lon'], row['lat'], shop_location[1], shop_location[0]) for _, row in df_drivers.iterrows()])

    # Add the distances as a new column in the drivers DataFrame
    df_drivers['distance'] = distances

    # Sort the drivers by their distances to the shop location
    df_drivers.sort_values('distance', inplace=True)

    # Get the ID of the closest available driver to the shop location
    closest_driver_id = df_drivers.loc[(df_drivers['disponibility'] == True) & (df_drivers['distance'] > 0), 'driver_id'].values[0]

    return closest_driver_id

In [177]:
# Ask the user to input a shop name
shop_name = input("Enter a shop name: ")
closest_driver_id = find_closest_driver(df_drivers, df_shops, shop_name)
print(f'The closest available driver to {shop_name} is {closest_driver_id}')

The closest available driver to La Mayor is B402
